In [1]:
import numpy as  np
import os
import cv2
import pandas as pd
#from PIL import Image as pil_image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import time

In [2]:
print("numpy version ", np.__version__)
print("pandas", pd.__version__)
print("openCV version ", cv2.__version__)

numpy version  1.16.6
pandas 1.1.0
openCV version  3.4.3


In [3]:
os.getcwd()

'C:\\Users\\TemwekaC'

In [4]:
path = "C:\\Users\\TemwekaC\\Isolation_and_Detection_Prey\\isolation"

In [23]:
# create dict to map class names to numbers for yolo
classes = {}
with open(os.path.join(path, "classes.txt"), "r") as myFile:
    for num, line in enumerate(myFile, 0):
        line = line.rstrip("\n")
        classes[num] = line
    myFile.close()

classes

{0: 'Jellyfish', 1: 'Fish'}

In [24]:
def BB_coordinates(filename):    
    coordinates = []
    with open(filename) as f:
        for line in f:
            labels = line.split()
            coord = [int(labels[0]), int(float(labels[-4])), int(float(labels[-3])) , int(float(labels[-2])), int(float(labels[-1]))]
            coordinates.append(coord)
        f.close()
    
    return coordinates

In [25]:
#directory to the images
image_dir = path + '\\data\\test\\test'

#Directory to the ground truth text files
gt_dir = path + '\\data\\test\\test\\txt'

#Directory to the prediction text files
pred_dir = path + '\\results\\main_data\\detection_files\\test'

#Directory of where to save new images
output_dir = path + '\\results\\main_data\\labelled_images\\test'

In [26]:
for filename in os.listdir(pred_dir):
    filename_str = filename[:-4]
    
    os.chdir(image_dir)
    image = cv2.imread((filename_str+".png"))
    
    os.chdir(gt_dir)
    gt_coordinates = BB_coordinates(filename)
    
    #Adding bounding box from ground truth in blue
    for row in gt_coordinates:
        cv2.rectangle(image,(row[1],row[2]),(row[3],row[4]),(255,0,0),2)
    
    os.chdir(pred_dir)
    pr_coordinates = BB_coordinates(filename)
    
    #Adding bounding box from YOLO Predictions in red
    for row in pr_coordinates:
        cv2.rectangle(image,(row[1],row[2]),(row[3],row[4]),(0,0,255),2)
        position = (row[1],row[2])
        cv2.putText(image, classes[row[0]], #text
                    position, cv2.FONT_HERSHEY_PLAIN, 
                     0.85, (0, 0, 0, 0), 1) 
    
    os.chdir(output_dir)
    cv2.imwrite((filename_str+".png"), image)

Images without predicted bounding boxes

In [27]:
filenames = []
os.chdir(path + '\\results\\main_data\\detection_files')
with open("No_predictions_test.txt") as f:
    for line in f:
        filenames.append(line.split())
    f.close()

In [28]:
#directory to the images
image_dir = path + '\\data\\test\\test'

#Directory to the ground truth text files
gt_dir = path + '\\data\\test\\test\\txt'

#Directory of where to save new images
output_dir = path + '\\results\\main_data\\labelled_images\\missed_test'

In [29]:
for file in filenames:
    filename = file[0]
    
    os.chdir(image_dir)
    image = cv2.imread((filename+".png"))
    
    os.chdir(gt_dir)
    gt_coordinates = BB_coordinates((filename+".txt"))
    
    #Adding bounding box with label from ground truth
    for row in gt_coordinates:
        cv2.rectangle(image,(row[1],row[2]),(row[3],row[4]),(255,0,0),2)
        position = (row[1],row[2])
        cv2.putText(image, classes[row[0]], #text
                    position, cv2.FONT_HERSHEY_PLAIN, 
                     0.85, (0, 0, 0, 0), 1) 
    
    os.chdir(output_dir)
    cv2.imwrite((filename+".png"), image)